In [0]:
import pandas as pd

In [0]:
# Lendo os dados
caminho_csv = "dbfs:/FileStore/tables/flights.csv"

spark_df = spark.read\
    .format("csv")\
    .option("header", True)\
    .option("sep", ",")\
    .option("inferSchema", True)\
    .load(caminho_csv)

In [0]:
# Convertendo Spark para Pandas
df = spark_df.toPandas()

In [0]:
# 1. Volume de voos por meses e dia da semana
voos_por_mes = df.groupby('MONTH').size().reset_index(name='total_voos')
voos_por_dia = df.groupby('DAY_OF_WEEK').size().reset_index(name='to  tal_voos')

In [0]:
display(voos_por_mes)

MONTH,total_voos
1,469968
2,429191
3,504312
4,485151
5,496993
6,503897
7,520718
8,510536
9,464946
10,486165


In [0]:
display(voos_por_dia)

DAY_OF_WEEK,to tal_voos
1,865543
2,844600
3,855897
4,872521
5,862209
6,700545
7,817764


In [0]:
# 2. Análise de atrasos
total_voos = len(df)
voos_atrasados = df[df['DEPARTURE_DELAY'] > 0]
perc_atrasos = (len(voos_atrasados) / total_voos) * 100
tempo_medio_atraso = voos_atrasados['DEPARTURE_DELAY'].mean()

print(f"Total de voos atrasados: {total_voos_atrasados}")
print(f"Porcentagem de voos atrasados: {porcentagem_voos_atrasados:.2f}%")
print(f"Média de tempo de atraso: {media_tempo_atraso:.2f} minutos")

Total de voos atrasados: 2125618
Porcentagem de voos atrasados: 36.53%
Média de tempo de atraso: 32.67 minutos


In [0]:
# 3. Atrasos por mes (geral e BOS)
atrasos_mes = df.groupby('MONTH').apply(
    lambda x: (x['DEPARTURE_DELAY'] > 0).mean() * 100
).reset_index(name='perc_atrasos')

atrasos_bos = df[df['ORIGIN_AIRPORT'] == 'BOS'].groupby('MONTH').apply(
    lambda x: (x['DEPARTURE_DELAY'] > 0).mean() * 100
).reset_index(name='perc_atrasos_bos')

In [0]:
display(atrasos_mes)

MONTH,perc_atrasos
1,37.58277159295952
2,40.41137861697939
3,38.43196275321626
4,34.48699477070026
5,35.98762960444111
6,42.74306058579432
7,40.255762235989536
8,37.380321857812184
9,28.5175052586752
10,29.846245616200264


In [0]:
display(atrasos_bos)

MONTH,perc_atrasos_bos
1,34.129229376485235
2,48.63961813842482
3,37.42854277404473
4,33.29
5,31.52035933990821
6,37.38619119878604
7,39.63273968810556
8,39.88999720331873
9,29.868393995476044
11,28.835587489433646


In [0]:
# 4. Analise de cancelamentos
total_cancelados = df['CANCELLED'].sum()
perc_cancelados = (total_cancelados / total_voos) * 100

cancelamentos_motivo = df[df['CANCELLED'] == 1]['CANCELLATION_REASON'].value_counts()

In [0]:
display(cancelamentos_motivo)

B    48851
A    25262
C    15749
D       22
Name: CANCELLATION_REASON, dtype: int64

In [0]:
# 5. Confiabilidade das companhias aéreas
confiabilidade_airlines = df.groupby('AIRLINE').agg({
    'DEPARTURE_DELAY': lambda x: (x <= 0).mean() * 100
}).reset_index()
confiabilidade_airlines.columns = ['AIRLINE', 'PONTUALIDADE']
confiabilidade_airlines = confiabilidade_airlines.sort_values('PONTUALIDADE', ascending=False)

In [0]:
display(confiabilidade_airlines)

AIRLINE,PONTUALIDADE
AS,74.3932622695208
HA,73.38603943780156
OO,69.26352036957405
EV,67.7294716395939
DL,67.32809593997358
US,66.55763279068012
AA,64.69756909243178
MQ,63.31830894132341
VX,61.39605511849183
F9,60.98573252895327


In [0]:
# Salvando os dados processados para o Power BI
voos_por_mes.to_csv('voos_por_mes.csv', index=False)
voos_por_dia.to_csv('voos_por_dia.csv', index=False)
atrasos_mes.to_csv('atrasos_mensais.csv', index=False)
atrasos_bos.to_csv('atrasos_bos.csv', index=False)
confiabilidade_airlines.to_csv('confiabilidade_airlines.csv', index=False)
